In [7]:
import pandas as pd
import numpy as np
from math import *
from random import *
import datetime as datetime
import matplotlib.pyplot as plt
import math
from numpy import unravel_index
from PointProcess import PointProcessTrain
from PointProcess import PointProcessRun
from numpy import array

In [8]:
# real data
fields = ['XCOORD', 'YCOORD', 'CALL_TYPE_FINAL_D', 'CALL_TYPE_FINAL', 'DATE_TIME']
full_data = pd.read_csv('FixedLatLongTimeLoc_AndTimeSorted.csv', usecols=fields)
full_data['DATE_TIME'] =  pd.to_datetime(full_data['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
full_data = full_data.sort_values(by='DATE_TIME')

In [358]:
# trained parameters for testing
trained = np.load('50by50w2_75_5.npz')
theta = trained['theta'][-1]
mu = trained['mu'][-1]
w = trained['w']
xsize = trained['xsize']
ysize = trained['ysize']

In [359]:
def ESTProcess(mu, theta, w, T):
    # mu units: #/day, k0 units: #/day, w units: #/day, T units: days?
    
    # base number of events
    p = np.random.poisson(mu*T)
    
    times = np.random.uniform(0, T, p) #(low, high, size)
    counts = 0
    countf = p-1
    
    while(countf>=counts):
        p=pois(theta)    #each event generates p offspring according to a Poisson r.v. with parameter theta
        for j in range(0, p):
            temp=times[counts]-np.log(np.random.rand())/w    
            if(temp < T):    
                times = np.append(times, temp)
                countf=countf+1
        counts=counts+1

    times = times[0:countf]
    
    times = np.sort(times)
    
    return times 


In [374]:
test = ESTProcess(mu[19][23], sum(theta), sum(w), 700)

In [375]:
for i in range(0, len(test)):
    date = full_data.DATE_TIME[0] + datetime.timedelta(days = test[i])
    print(date)

In [ ]:
# Todo: poisson process for each grid cell of importance. 
# for x in range(0, xsize):
    # for y in range(0, ysize):
        #...
# Generate array in form of [[event time, x, y], [event time, x, y]...]. Sort by event time. 
# Should produce list of potential events with corresponding grid locations